In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
!pip install datatable > /dev/null
import datatable as dt
from sklearn import impute
import gc

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

The system cannot find the path specified.


ModuleNotFoundError: No module named 'datatable'

In [2]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

ValueError: Please provide a TPU Name to connect to.

In [5]:
SEED = 42

tf.random.set_seed(SEED)
np.random.seed(SEED)

# df = pd.read_csv("../input/jane-street-market-prediction/train.csv")
df = dt.fread('../input/jane-street-market-prediction/train.csv')
df = df.to_pandas()

In [6]:
df = df.set_index('ts_id', drop=True)
df.drop(columns=[f"resp_{i}" for i in range(1, 5)], inplace=True)
print(f'Done loading data. df shape is {df.shape}')
df.head()

Done loading data. df shape is (2390491, 133)


,date,weight,resp,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
ts_id,,,,,,,,,,,,,,,,,,,,,
0,0,0.000000,0.006270,1,-1.872746,-2.191242,-0.474163,-0.323046,0.014688,-0.002484,...,NaN,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807
1,0,16.673515,-0.009792,-1,-1.349537,-1.704709,0.068058,0.028432,0.193794,0.138212,...,NaN,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684
2,0,0.000000,0.023970,-1,0.812780,-0.256156,0.806463,0.400221,-0.614188,-0.354800,...,NaN,NaN,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299
3,0,0.000000,-0.003200,-1,1.174378,0.344640,0.066872,0.009357,-1.006373,-0.676458,...,NaN,NaN,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469
4,0,0.138531,-0.002604,1,-3.172026,-3.093182,-0.161518,-0.128149,-0.195006,-0.143780,...,NaN,NaN,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633


In [7]:
TARGET = 'resp'
FEATURES = [f"feature_{i}" for i in range(1, 130)]
train_pos, train_neg = df.loc[df.feature_0 > 0], df.loc[df.feature_0 < 0]
train_pos.drop(columns=[TARGET, 'feature_0'], inplace=True)
train_neg.drop(columns=[TARGET, 'feature_0'], inplace=True)
gc.collect()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


52

### Null Values 🈳

In [ ]:
#df.fillna(df.mean(),inplace=True) 
# Instead of imputing with median, let's try K-Nearest Neighbors Imputing
# with tpu_strategy.scope():
#     knn_imputer = impute.KNNImputer(n_neighbors=2)
# df = knn_imputer.fit_transform(df)

In [8]:
nan_neg = pd.read_csv("../input/janestreetimputeddata/nan_neg.csv", header=None, sep=' ').values.astype(int)
nan_pos = pd.read_csv("../input/janestreetimputeddata/nan_pos.csv", header=None, sep=' ').values.astype(int)

In [9]:
gc.collect()

20

In [10]:
# Split into X and y
from copy import deepcopy as dc
X_pos = dc(train_pos[FEATURES].values)
X_neg = dc(train_neg[FEATURES].values)
del train_pos, train_neg
gc.collect()

20

In [11]:
file = 'knn_5'
path = "../input/janestreetimputeddata/"
X_pos[nan_pos[0], nan_pos[1]] = pd.read_csv(path+f"positive_{file}.csv", 
                                            header=None, sep=' ').values.flatten()
X_neg[nan_neg[0], nan_neg[1]] = pd.read_csv(path+f"negative_{file}.csv",                                         
                                            header=None, sep=' ').values.flatten()

In [12]:
print("df.shape: "+ str(df.shape))
print(type(X_pos))
print(X_neg.shape)

df.shape: (2390491, 133)
<class 'numpy.ndarray'>
(1183486, 129)


In [13]:
df = np.concatenate((X_pos, X_neg), axis=0)
print(df.shape)


(2390491, 129)


In [14]:
del X_pos, X_neg, nan_neg, nan_pos
gc.collect()

80

In [15]:
df = pd.DataFrame(df, columns = FEATURES)

In [16]:
df.head(5)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
0,-1.872746,-2.191242,-0.474163,-0.323046,0.014688,-0.002484,-0.541212,-0.539259,-0.989982,-1.055090,...,0.603878,6.086305,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807
1,-3.172026,-3.093182,-0.161518,-0.128149,-0.195006,-0.143780,0.326871,0.081905,2.683018,1.450991,...,0.745019,5.354213,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633
2,-3.172026,-3.093182,-0.363836,-0.291496,0.128422,0.096168,-1.290501,-1.173106,-3.727364,-2.451716,...,1.120067,4.167835,1.537913,4.785838,1.637435,6.968002,2.354338,5.825499,1.778029,4.740577
3,2.744408,1.412127,-0.250801,-0.161357,0.014279,-0.008666,-1.418110,-1.241481,0.711444,-0.278608,...,1.430190,3.332330,1.796860,3.177064,0.999252,2.906432,1.589816,2.435999,1.472419,2.245991
4,-3.172026,-3.093182,0.151195,0.045958,0.770927,0.434973,-0.234912,-0.327942,-2.349983,-1.614583,...,1.581096,6.305170,2.324290,4.881133,2.115830,6.337250,3.059392,5.350729,2.755876,4.968388


In [17]:
df.to_csv('imputed_df.csv', index=False)

In [18]:
# df = pd.read_csv("../input/jane-street-market-prediction/train.csv")
train = dt.fread('../input/jane-street-market-prediction/train.csv')
train = train.to_pandas()
gc.collect()

0

In [20]:
upper = train[train['feature_0'] == 1]

In [21]:
lower = train[train['feature_0'] == -1]

In [24]:
train = pd.concat((upper, lower), axis = 0)
del upper, lower
gc.collect()
df.to_csv('train.csv', index=False)

In [ ]:
df = dt.fread('./imputed_df.csv')
df = df.to_pandas()

In [ ]:
df = pd.concat((train, df), axis = 1)


In [22]:
del train
gc.collect()

78

In [ ]:
# 2, 10 , 6
#with tpu_strategy.scope():
#df = train.merge(df, how="left", on=['feature_2','feature_10','feature_6'])
#df.head(5)

In [ ]:
#df.shape

In [ ]:
# df = df.query('date > 85').reset_index(drop = True) 
# df = df[df['weight'] != 0]

In [ ]:
# from tsfresh.feature_extraction import feature_calculators as fc
# df['count_above_mean'] = fc.count_above_mean(df.to_numpy())
# df['mean_abs_change'] = fc.mean_abs_change(df.to_numpy())
# df['abs_energy'] = fc.abs_energy(df.to_numpy())

In [ ]:
# df['action'] = ((df['resp'].values) > 0).astype(int)


# features = [c for c in df.columns if "feature" in c]

# f_mean = np.mean(df[features[1:]].values,axis=0)

# resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

# X_train = df.loc[:, train.columns.str.contains('feature')]

# y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

In [ ]:
# test = dt.fread('../input/jane-street-market-prediction/example_test.csv')
# test = test.to_pandas()

In [ ]:
# def create_mlp(
#     num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
# ):

#     inp = tf.keras.layers.Input(shape=(num_columns,))
#     x = tf.keras.layers.BatchNormalization()(inp)
#     x = tf.keras.layers.Dropout(dropout_rates[0])(x)
#     for i in range(len(hidden_units)):
#         x = tf.keras.layers.Dense(hidden_units[i])(x)
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
#         x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

#     x = tf.keras.layers.Dense(num_labels)(x)
#     out = tf.keras.layers.Activation("sigmoid")(x)

#     model = tf.keras.models.Model(inputs=inp, outputs=out)
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#         loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
#         metrics=tf.keras.metrics.AUC(name="AUC"),
#     )

#     return model

In [ ]:
# batch_size = 5000
# hidden_units = [145, 145, 145]
# dropout_rates = [0.2, 0.2, 0.2, 0.2]
# label_smoothing = 1e-2
# learning_rate = 1e-3

# with tpu_strategy.scope():
#     clf = create_mlp(
#         len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
#         )

#     clf.fit(X_train, y_train, epochs=200, batch_size=5000)


# models = []

# models.append(clf)

In [ ]:
# th = 0.5


# f = np.median
# models = models[-3:]

In [ ]:

# import janestreet
# env = janestreet.make_env()
# for (test_df, pred_df) in tqdm(env.iter_test()):
#     if test_df['weight'].item() > 0:
#         x_tt = test_df.loc[:, features].values
#         if np.isnan(x_tt[:, 1:].sum()):
#             x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
#         pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
#         pred = f(pred)
#         pred_df.action = np.where(pred >= th, 1, 0).astype(int)
#     else:
#         pred_df.action = 0
#     env.predict(pred_df)

### Reference
[Imputing-missing-values](https://www.kaggle.com/louise2001/imputing-missing-values)